# Assignment 3

In [3]:
# importing necessary packages
import pandas as pd
import tarfile

## Problem 1: Spam and Ham data

In [4]:
def read_data(fn):
    dataset = list()
    with tarfile.open(fn) as tf:
        for i in tf:
            if i.isfile():
                with tf.extractfile(i) as f:
                    b  = f.read()
                    if b'charset=utf-8' in b or b'charset=UTF-8' in b:
                        s = b.decode('utf-8')
                    else:
                        s = b.decode('iso-8859-1')
                    dataset.append(s)
    return dataset

In [5]:
data = {
    'easy_ham': read_data('data/20021010_easy_ham.tar.bz2'),
    'hard_ham': read_data('data/20021010_hard_ham.tar.bz2'),
    'spam': read_data('data/20021010_spam.tar.bz2')
}

In [14]:
df_eh = pd.DataFrame(data['easy_ham']) 
df_hh = pd.DataFrame(data['hard_ham'])
df_spam = pd.DataFrame(data['spam'])

In [16]:
df_eh.to_csv("eh_content.csv")
df_hh.to_csv("hh_content.csv")
df_spam.to_csv("spam_content.csv")



B. Data Splitting